In [1]:
import pandas as pd
import numpy as np
import xlrd
import os
from datetime import datetime, date
from time import mktime
import time

In [2]:
# Import Raw Data from excel files into dataFrames
df10=pd.read_excel('orig_data/mlb_odds_2010.xlsx')
df10['Year'] = 2010
df11=pd.read_excel('orig_data/mlb_odds_2011.xlsx')
df11['Year'] = 2011
df12=pd.read_excel('orig_data/mlb_odds_2012.xlsx')
df12['Year'] = 2012
df13=pd.read_excel('orig_data/mlb_odds_2013.xlsx')
df13['Year'] = 2013
df14=pd.read_excel('orig_data/mlb_odds_2014.xlsx')
df14['Year'] = 2014
df15=pd.read_excel('orig_data/mlb_odds_2015.xlsx')
df15['Year'] = 2015
df16=pd.read_excel('orig_data/mlb_odds_2016.xlsx')
df16['Year'] = 2016
df17=pd.read_excel('orig_data/mlb_odds_2017.xlsx')
df17['Year'] = 2017
df18=pd.read_excel('orig_data/mlb_odds_2018.xlsx')
df18['Year'] = 2018
df19=pd.read_excel('orig_data/mlb_odds_2019.xlsx')
df19['Year'] = 2019

In [3]:
# make list of dataFrames
dfList = [df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

In [4]:
# reformat data
for df in dfList:
    try:
        dfnew=df['RL'].str.split(pat="(", n=1, expand=True)
        dfnewer = dfnew[1].str.rstrip("\)")
        df['run_line_close'] = dfnew[0].apply(float)
        df['run_line_odds_close'] = dfnewer.apply(int)
    except KeyError:
        pass

In [5]:
for df in dfList:
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.rename({"vh": "visitor_home", "final": "team_run_total", "close":"money_line_close", "open":"money_line_open", "openou":"over_under_line_open", "open_ou":"over_under_line_open", "unnamed:_18":"over_under_odds_open", "unnamed:_19":"over_under_odds_open", "close_ou":"over_under_line_close", "closeou":"over_under_line_close", "unnamed:_20":"over_under_odds_close", "unnamed:_21":"over_under_odds_close"}, axis='columns', inplace=True)    

In [6]:
def convt(num):
    if type(num) == str:
        num = float(num[0]) + .5
    return num
for df in dfList:
    df['over_under_line_close'] = df['over_under_line_close'].apply(convt)

In [7]:
# create additional columns
for df in dfList:
    totRuns = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                totRuns.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                counter += row.team_run_total
            else:
                counter += row.team_run_total
                totRuns.append(counter)
                totRuns.append(counter)
    except TypeError:
        print(row)
    df['total_runs_game'] = totRuns

In [8]:
for df in dfList:
    RunDif = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                RunDif.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                team1 = row.team_run_total
            else:
                team2 = row.team_run_total
                dif = team1 - team2
                RunDif.append(dif)
                RunDif.append(-dif)
    except TypeError:
        print(row)
    df['run_dif_game'] = RunDif

In [9]:
for df in dfList:
    df['date'] = df['year'].astype(str) + '-' + df['date'].astype(str).apply(lambda x: '0' + x if len(x) == 3 else x)
    df['date'] = df['date'].apply(lambda x: x[:7] + '-' + x[7:])

In [10]:
df1 = pd.concat(dfList, sort=True, ignore_index=True)

In [11]:
df1.drop(['year', 'rl', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th'], axis=1, inplace=True)

In [12]:
df1 = df1[df1['visitor_home'] != 'N']

In [13]:
colNew = []
i = 0
for index, row in df1.iterrows():
    if index == 0:
        pass
    elif index % 2 == 0:
        i += 1
    colNew.append(i)
df1['gameno'] = colNew
df1.replace('LOS','LAD',inplace=True)
df1.replace('CUB','CHC',inplace=True)
df1.replace('SDG','SD',inplace=True)
df1.replace('SFO','SF',inplace=True)
df1.replace('KAN','KC',inplace=True)
df1.replace('TAM','TB',inplace=True)

In [14]:
df1 = df1.reindex(columns=['date', 'gameno', 'team', 'visitor_home', 'team_run_total', 'total_runs_game', 'money_line_close', 'money_line_open', 'over_under_line_close', 'over_under_line_open', 'over_under_odds_close', 'over_under_odds_open', 'pitcher', 'rot', 'run_dif_game', 'run_line_close', 'run_line_odds_close'])

In [15]:
df1.replace(['NL', 'nan'], np.nan, inplace=True)

In [16]:
#merge visitor and home rows together
df_v = df1.loc[df1['visitor_home'] == 'V']
df_h = df1.loc[df1['visitor_home'] == 'H']
df_h.columns = ['h_date', 'gameno', 'h_team', 'home', 'h_team_run_total',
                   'h_total_runs_game', 'h_money_line_close', 'h_money_line_open',
                   'under_line_close', 'under_line_open',
                   'under_odds_close', 'under_odds_open', 'h_pitcher', 'h_rot',
                   'h_run_dif_game', 'h_run_line_close', 'h_run_line_odds_close']
df_v.columns = ['date', 'gameno', 'v_team', 'visitor', 'v_team_run_total',
                   'total_runs_game', 'v_money_line_close', 'v_money_line_open',
                   'over_line_close', 'over_line_open',
                   'over_odds_close', 'over_odds_open', 'v_pitcher', 'v_rot',
                   'v_run_dif_game', 'v_run_line_close', 'v_run_line_odds_close']

df3 = pd.merge(df_v, df_h, on='gameno', how='right')

In [17]:
#drop unnecessary columns
df3.drop(['h_total_runs_game','h_date'], axis=1, inplace=True)

In [18]:
#reindex columns
df_final = df3.reindex(columns = ['date', 'gameno', 
             
             'visitor', 'v_team', 'v_team_run_total', 
             'v_money_line_close', 'v_money_line_open', 
             'v_run_line_close', 'v_run_line_odds_close', 
             'v_pitcher', 'v_rot', 'v_run_dif_game',
             
             'home', 'h_team', 'h_team_run_total', 
             'h_money_line_close', 'h_money_line_open', 
             'h_run_line_close', 'h_run_line_odds_close',
             'h_pitcher', 'h_rot', 'h_run_dif_game',
             
             'total_runs_game', 
             'over_line_close', 'over_line_open', 'over_odds_close', 'over_odds_open', 
             'under_line_close', 'under_line_open', 'under_odds_close', 'under_odds_open' 
            ])

In [78]:
unix_dict = {}
for index,row in df_final.iterrows():
    dt = row['date']
    yr = int(dt[0:4])
    mon = int(dt[5:7])
    day = int(dt[8:10])
    
    dt2 = date(yr,mon,day)
    unix = dt2.strftime("%s")
    unix_dict[index] = unix

In [79]:
pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', 'dataset.pickle'))
df_final.to_pickle(pickle_path)

In [80]:
df_final.head()

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,...,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open
0,2010-04-04,0,V,NYY,7.0,107.0,104,NaN,NaN,CSABATHIA-L,...,2.0,16.0,9.0,9.0,-105,-116,9.0,9.0,-115,-104
1,2010-04-05,1,V,PHI,11.0,-200.0,-200,NaN,NaN,RHALLADAY-R,...,-10.0,12.0,7.5,7.5,-125,100,7.5,7.5,105,-120
2,2010-04-05,2,V,MIA,1.0,-102.0,105,NaN,NaN,JJOHNSON-R,...,6.0,8.0,7.0,7.0,-130,-125,7.0,7.0,110,105
3,2010-04-05,3,V,STL,11.0,-143.0,-155,NaN,NaN,CARPENTER-R,...,-5.0,17.0,7.5,7.5,-105,-105,7.5,7.5,-115,-115
4,2010-04-05,4,V,LAD,5.0,-126.0,-155,NaN,NaN,VPADILLA-R,...,6.0,16.0,8.5,8.5,-120,-110,8.5,8.5,100,-110


In [81]:
df_u = pd.DataFrame.from_dict(unix_dict, orient='index')

In [82]:
df_u.head()

,0
0,1270353600
1,1270440000
2,1270440000
3,1270440000
4,1270440000


In [83]:
df_unix_date = pd.merge(df_final, df_u, how='outer', left_index=True, right_index=True)

In [84]:
df_unix_date.drop(['date'], axis=1, inplace=True)

In [85]:
df_unix_date = df_unix_date.rename(columns={0:'date'})

In [86]:
df_final_unix = df_unix_date.reindex(columns = ['date', 'gameno', 
             
             'visitor', 'v_team', 'v_team_run_total', 
             'v_money_line_close', 'v_money_line_open', 
             'v_run_line_close', 'v_run_line_odds_close', 
             'v_pitcher', 'v_rot', 'v_run_dif_game',
             
             'home', 'h_team', 'h_team_run_total', 
             'h_money_line_close', 'h_money_line_open', 
             'h_run_line_close', 'h_run_line_odds_close',
             'h_pitcher', 'h_rot', 'h_run_dif_game',
             
             'total_runs_game', 
             'over_line_close', 'over_line_open', 'over_odds_close', 'over_odds_open', 
             'under_line_close', 'under_line_open', 'under_odds_close', 'under_odds_open' 
            ])

In [87]:
df_final_unix

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,...,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open
0,1270353600,0,V,NYY,7.0,107.0,104,NaN,NaN,CSABATHIA-L,...,2.0,16.0,9.0,9.0,-105,-116,9.0,9.0,-115,-104
1,1270440000,1,V,PHI,11.0,-200.0,-200,NaN,NaN,RHALLADAY-R,...,-10.0,12.0,7.5,7.5,-125,100,7.5,7.5,105,-120
2,1270440000,2,V,MIA,1.0,-102.0,105,NaN,NaN,JJOHNSON-R,...,6.0,8.0,7.0,7.0,-130,-125,7.0,7.0,110,105
3,1270440000,3,V,STL,11.0,-143.0,-155,NaN,NaN,CARPENTER-R,...,-5.0,17.0,7.5,7.5,-105,-105,7.5,7.5,-115,-115
4,1270440000,4,V,LAD,5.0,-126.0,-155,NaN,NaN,VPADILLA-R,...,6.0,16.0,8.5,8.5,-120,-110,8.5,8.5,100,-110
5,1270440000,5,V,COL,5.0,-105.0,110,NaN,NaN,UJIMENEZ-R,...,-2.0,8.0,7.5,7.5,105,-105,7.5,7.5,-125,-115
6,1270440000,6,V,CHC,5.0,110.0,115,NaN,NaN,CZAMBRANO-R,...,11.0,21.0,8.5,8.5,100,-110,8.5,8.5,-120,-110
7,1270440000,7,V,SD,3.0,166.0,155,NaN,NaN,JGARLAND-R,...,3.0,9.0,8.0,8.0,-120,-110,8.0,8.0,100,-110
8,1270440000,8,V,SF,5.0,-147.0,-130,NaN,NaN,TLINCECUM-R,...,-3.0,7.0,7.0,7.5,-110,105,7.0,7.5,-110,-125
9,1270440000,9,V,CLE,0.0,127.0,150,NaN,NaN,WESTBROOK-R,...,6.0,6.0,9.0,9.0,-105,-105,9.0,9.0,-115,-115


In [71]:
pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', 'unix_dataset.pickle'))
df_final_unix.to_pickle(pickle_path)

In [68]:
day_early = 24*60*60
sd = 1270354000 - day_early
ed = 1554610000
df_filter = df_final_unix[df_final_unix['date'].between(sd, ed, inclusive=True)]

In [91]:
df_filter.head()

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,v_rot,v_run_dif_game,home,h_team,h_team_run_total,...,h_money_line_open,h_run_line_close,h_run_line_odds_close,h_pitcher,h_rot,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open
0,1.270354e+09,0,V,NYY,7.0,107.0,104,NaN,NaN,CSABATHIA-L,931,-2.0,H,BOS,9.0,...,-114,NaN,NaN,JBECKETT-R,932,2.0,16.0,9.0,9.0,-105,-116,9.0,9.0,-115,-104
1,1.270440e+09,1,V,PHI,11.0,-200.0,-200,NaN,NaN,RHALLADAY-R,901,10.0,H,WAS,1.0,...,175,NaN,NaN,JLANNAN-L,902,-10.0,12.0,7.5,7.5,-125,100,7.5,7.5,105,-120
2,1.270440e+09,2,V,MIA,1.0,-102.0,105,NaN,NaN,JJOHNSON-R,903,-6.0,H,NYM,7.0,...,-115,NaN,NaN,JOSANTANA-L,904,6.0,8.0,7.0,7.0,-130,-125,7.0,7.0,110,105
3,1.270440e+09,3,V,STL,11.0,-143.0,-155,NaN,NaN,CARPENTER-R,905,5.0,H,CIN,6.0,...,140,NaN,NaN,AHARANG-R,906,-5.0,17.0,7.5,7.5,-105,-105,7.5,7.5,-115,-115
4,1.270440e+09,4,V,LAD,5.0,-126.0,-155,NaN,NaN,VPADILLA-R,907,-6.0,H,PIT,11.0,...,140,NaN,NaN,ZDUKE-L,908,6.0,16.0,8.5,8.5,-120,-110,8.5,8.5,100,-110
